# Overview

Use keras to implement the tutorial convolutional neural network featured in the tensorflow tutorial

In [1]:
!conda list

# packages in environment at C:\Users\Ulric\Anaconda3:
#
# Name                    Version                   Build  Channel
_license                  1.1                      py35_1  
_nb_ext_conf              0.3.0                    py35_0  
_tflow_190_select         0.0.1                       gpu  
absl-py                   0.5.0                      py_0    conda-forge
alabaster                 0.7.9                    py35_0  
anaconda                  custom                   py35_0  
anaconda-clean            1.0.0                    py35_0  
anaconda-client           1.7.2                    py35_0  
anaconda-navigator        1.9.2                    py35_0  
anaconda-project          0.4.1                    py35_0  
appdirs                   1.4.3                     <pip>
argcomplete               1.0.0                    py35_1  
astor                     0.7.1                      py_0    conda-forge
astroid                   1.4.9                    py35_0  
astropy     

# Verify tensor flow

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

# Startup keras

In [3]:
import keras

import numpy as np

Using TensorFlow backend.


# Load data

In [4]:
from keras.datasets import mnist

In [5]:
batch_size = 128
num_classes = 10
epochs = 10

img_x, img_y = (28 , 28)

input_shape = (img_x, img_y , 1)

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [8]:
# reshape the data into a 4D tensor - (sample_number, x_img_size, y_img_size, num_channels)
# because the MNIST is greyscale, we only have a single channel - RGB colour images would have 3
x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1)
x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 1)

In [9]:
# convert the data to the right type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [10]:
# convert class vectors to binary class matrices - this is for use in the
# categorical_crossentropy loss below
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [11]:
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (60000, 10)
y_test shape: (10000, 10)


# Setup a model

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

### Specify model

In [13]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

### Compile model

In [14]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

# Train the model

### Eval function

In [15]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()

# Fit

In [16]:
%%time

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s - loss: 0.1509 - acc: 0.9557 - val_loss: 0.0447 - val_acc: 0.9855
Epoch 2/10
60000/60000 [==============================] - 6s - loss: 0.0411 - acc: 0.9873 - val_loss: 0.0487 - val_acc: 0.9849
Epoch 3/10
60000/60000 [==============================] - 7s - loss: 0.0271 - acc: 0.9917 - val_loss: 0.0341 - val_acc: 0.9889
Epoch 4/10
60000/60000 [==============================] - 6s - loss: 0.0205 - acc: 0.9935 - val_loss: 0.0321 - val_acc: 0.9903
Epoch 5/10
60000/60000 [==============================] - 6s - loss: 0.0158 - acc: 0.9948 - val_loss: 0.0259 - val_acc: 0.9927
Epoch 6/10
60000/60000 [==============================] - 7s - loss: 0.0125 - acc: 0.9961 - val_loss: 0.0220 - val_acc: 0.9936
Epoch 7/10
60000/60000 [==============================] - 7s - loss: 0.0102 - acc: 0.9965 - val_loss: 0.0211 - val_acc: 0.9935
Epoch 8/10
60000/60000 [==============================] - 7s 

# Use model for prediction

In [18]:
preds_test = model.predict(x_test)

In [19]:
preds_test.shape

(10000, 10)

In [20]:
preds_test[0]

array([  1.15235042e-15,   1.43304041e-15,   2.52884817e-13,
         2.01078862e-10,   6.61074929e-18,   8.05289999e-16,
         6.88082044e-22,   1.00000000e+00,   3.25479391e-17,
         3.03041620e-11], dtype=float32)

In [23]:
y_test.shape

(10000, 10)

In [24]:
y_test[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])

# Save model

In [25]:
model.save("./models/minist_cnn.h5")